In [ ]:
from nba_api.stats.endpoints import commonplayerinfo # if you don't have it : "pip install nba_api" in the shell 
from nba_api.stats.static import players
import pandas as pd
import time
import json
import numpy as np
from tqdm import tqdm # to see the progression in the for loop
pd.set_option('display.max_columns',None)

Get the ID list of all players

In [ ]:
nba_players = players.get_players()
id_list = []
for player in nba_players:
   id_list.append(player['id'])
print(len(id_list))

Get the headers columns for both Dataframes and create them. Here I use player id 893 (Michael Jordan) to get these infos but you can use the player id you like the most.

In [ ]:
player_info = commonplayerinfo.CommonPlayerInfo(893)
r = json.loads(player_info.get_json())
headers_CommonPlayerInfo = r['resultSets'][0]['headers']
hearders_PlayerHeadlineStats = r['resultSets'][1]['headers']
df_CommonPlayerInfo = pd.DataFrame(columns=headers_CommonPlayerInfo)
df_PlayerHeadlineStats = pd.DataFrame(columns=hearders_PlayerHeadlineStats)

Example of extracting information for the first 10 players in the id list. I extract these infos in temp dataframe and add them in the dataframes I want to save (in csv).
I also introduce a random lag between each player I request via the nba_api. If you are to demanding, it can shut down the connexion...

In [ ]:

for player_id in tqdm(id_list[:10]):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id)
    r = json.loads(player_info.get_json())
    temp_CommonPlayerInfo = pd.DataFrame(r['resultSets'][0]['rowSet'],columns=headers_CommonPlayerInfo)
    temp_PlayerHeadlineStats = pd.DataFrame(r['resultSets'][1]['rowSet'],columns=hearders_PlayerHeadlineStats)
    df_CommonPlayerInfo = pd.concat([df_CommonPlayerInfo, temp_CommonPlayerInfo], axis=0)
    df_PlayerHeadlineStats = pd.concat([df_PlayerHeadlineStats, temp_PlayerHeadlineStats], axis=0)
    print(f'Finished scraping data for id {player_id}')
    lag = np.random.uniform(low=1, high=2)
    print(f'...waiting {round(lag,1)} seconds...')
    time.sleep(lag)

print('FINISHED')

Saving to csv

In [ ]:
df_PlayerHeadlineStats.to_csv('PlayerHeadlineStats_first10.csv',index=False,header=True)

In [ ]:
df_CommonPlayerInfo.to_csv('CommonPlayerInfo_first10.csv',index=False,header=True)